In [3]:
import os 
import json
import pprint as pp
import tweepy
import glob
import pandas as pd

In [4]:
#Twitter API credentials
api_key = "KLRpHp13jm62W7DK0QT4eX3d7"
api_secret = "30S9ZtJFYS6MLFUwdW7DVUgU6a0AXJ9ZqLw7Sey0Jr4ZU95XcU"
access_token = "1455882501130493955-uGFA6LQQvtxjQC9b7UoD7DCnjZhscy"
access_secret = "gEDVqti3J4gt0NzqM8PFdOYHrPQFkppwVS0redI8JzNbd"

In [5]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# JSON

In [6]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS e FOLLOWING

In [6]:
def findUsers(userlist,mode): #mode: 0 = followers, 1 = following
    quantity=20
    foundlist={}
    l2list = {}

    for user in userlist:
        print(f"Processing user {user}")

        found = []
        l2list[user]=[]

        if mode==0:
            usergroup=tweepy.Cursor(api.followers,screen_name = user ).items(quantity)
        else:
            usergroup=tweepy.Cursor(api.friends,screen_name = user ).items(quantity)

        for item in usergroup:

            json_data = item._json

            found_user = {}
            found_user["id"] = json_data["id"]
            found_user["name"] = json_data["name"]
            found_user["screen_name"] = json_data["screen_name"]
            found_user["location"] = json_data["location"]
            found_user["followers_count"] = json_data["followers_count"]
            found_user["friends_count"] = json_data["friends_count"]
            found_user["protected"] = json_data["protected"]


            if item.friends_count < 1000 and item.followers_count<1000 and json_data["protected"] == False and (found_user not in found):
                found.append(found_user)
                l2list[user].append(item.screen_name)
                print(item.screen_name)

            if len(found) >= 5:
                break

        if mode==0:
            serialize_json(data_folder, user+"_followers.json", found)
        else:
            serialize_json(data_folder, user+"_following.json", found)

    return l2list

l1_followers=findUsers(["mizzaro","Miccighel_"],0)
#l1_following=findUsers(["mizzaro","Miccighel_"],1)

l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0)
l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1)

l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0)
l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1)

#l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0)
#l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1)

#l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0)
#l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1)

Processing user mizzaro
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
Data serialized to path: data/mizzaro_followers.json
Processing user Miccighel_
Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Data serialized to path: data/Miccighel__followers.json
Processing user OlegZendel
bevan_koopman
Marwah_k
ivrik
Dars2022_Sympo
ameertawfik
Data serialized to path: data/OlegZendel_followers.json
Processing user LyndaSaidLhadj
libinfopro
Christo12656926
gael_dias
Nthakur20
AthmaniAmine
Data serialized to path: data/LyndaSaidLhadj_followers.json
Processing user acai2021
TilburgU_DCA
srj_krishna
gabriellearuta
lesleywilton
sencerdeniz
Data serialized to path: data/acai2021_followers.json
Processing user rodger_benham
csinschools
MaksudaBilkis
jparapar
ameertawfik
Jinxuan16807201
Data serialized to path: data/rodger_benham_followers.json
Processing user ColoursRiot
ciscaredda0111
AmbesiAntonio
BrillantVacant
LuvYahBoo24
Data serialized to path: data/ColoursRiot_followers.json


In [21]:
# Uniamo tutti i file json in un unico file
fileList=glob.glob(data_folder+"/*.json")

totalData=[]

for fileName in fileList:
    with open(fileName,'r',encoding="UTF-8") as file:
        data=json.load(file)
        for user in data:
            totalData.append(user)

serialize_json(data_folder, "total_users.json", totalData)

#with open(data_folder+"/total_users.json","w") as outfile:
#    json.dump(totalData,outfile)

Data serialized to path: data/total_users.json


# Dettagli del profilo dei followers di primo e secondo livello

In [11]:
def userDetails(userList):
    
    for user in userList:
        
        raw_data = api.get_user(user)
        json_data = raw_data._json
        profile_data = {}
        profile_data["id"] = json_data["id"]
        profile_data["name"] = json_data["name"]
        profile_data["screen_name"] = json_data["screen_name"]
        profile_data["location"] = json_data["location"]
        profile_data["description"] = json_data["description"]
        profile_data["followers_count"] = json_data["followers_count"]
        profile_data["friends_count"] = json_data["friends_count"]
        serialize_json(data_folder, "1_"+profile_data["screen_name"]+"_profile_data.json", profile_data)
        pp.pprint(profile_data)
        
userDetails(l1_followers)
userDetails(l2a_followers_mizzaro)
userDetails(l2a_followers_miccighel)
#userDetails(l2b_followers_mizzaro)
#userDetails(l2b_followers_miccighel)

Data serialized to path: data/1_mizzaro_profile_data.json
{'description': '',
 'followers_count': 162,
 'friends_count': 358,
 'id': 18932422,
 'location': '',
 'name': 'mizzaro',
 'screen_name': 'mizzaro'}
Data serialized to path: data/1_Miccighel__profile_data.json
{'description': 'University Life, Computer Science, Photography, Reading, '
                'Gaming, Comics. PhD Student at the University of Udine. '
                "That's me.",
 'followers_count': 330,
 'friends_count': 213,
 'id': 15750573,
 'location': 'Chiusaforte',
 'name': 'Michael Soprano',
 'screen_name': 'Miccighel_'}
Data serialized to path: data/1_OlegZendel_profile_data.json
{'description': 'IR Ph.D. candidate at @RMIT',
 'followers_count': 92,
 'friends_count': 514,
 'id': 998681467696885761,
 'location': 'Melbourne, Victoria',
 'name': 'Oleg Zendel',
 'screen_name': 'OlegZendel'}
Data serialized to path: data/1_LyndaSaidLhadj_profile_data.json
{'description': 'Assistant Professor @esi_ex_ini, former Post-d

# Ultimi 5 tweet di ciascun follower di primo e secondo livello

In [13]:
def getTweets(userList):
    
    for user in userList:
        
        tweets = []
        for item in tweepy.Cursor(
                api.user_timeline, 
                screen_name=user,
                tweet_mode="extended"
            ).items(5):
            raw_tweet = item._json
            tweet = {}
            tweet["id"]= raw_tweet["id"]
            tweet["created_at"] = raw_tweet["created_at"]
            tweet["full_text"] = raw_tweet["full_text"],
            tweet["favorite_count"]= raw_tweet["favorite_count"]
            tweets.append(tweet)
        serialize_json(data_folder, user+"_recent_tweets.json", tweets)
        pp.pprint(tweets)
        
getTweets(l1_followers)
getTweets(l2a_followers_mizzaro)
getTweets(l2a_followers_miccighel)
#getTweets(l2b_followers_mizzaro)
#getTweets(l2b_followers_miccighel)

Data serialized to path: data/mizzaro_recent_tweets.json
[{'created_at': 'Thu Nov 11 18:44:44 +0000 2021',
  'favorite_count': 0,
  'full_text': ('@JulioGonzalo1 @lintool But you did it!',),
  'id': 1458868311777763333},
 {'created_at': 'Mon Oct 18 07:49:11 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @eglu81: "We argue that bias is part of human nature, and '
                'that it should be managed rather than removed." \n'
                '\n'
                'From our position paper at #…',),
  'id': 1450006026745491459},
 {'created_at': 'Mon Oct 04 22:04:34 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @caslernoel: Anti-vaxxers react to FB outage: ‘how am I '
                'supposed to do my own research now?’',),
  'id': 1445147863453995013},
 {'created_at': 'Mon Oct 04 21:39:32 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @AcademicsSay: academia is wild because one day you get '
                'behind on writing and then you stay that way you

# Rete sociale

In [22]:
# Creazione dataframe dei Nodi
def create_node(follower_of_user):
    #lettura json dell'utente desiderato
    with open(data_folder+"/"+follower_of_user+".json", "r", encoding="UTF-8") as read_file:
        data = json.load(read_file)
    #aggiunta dei nodi 
    followers_nodes=[]
    for result in data:
        followers_nodes.append(result) 
    #df_nodi_grafo
    df_nodes = pd.DataFrame(followers_nodes)
    return df_nodes

print(create_node('total_users'))


                     id             name  screen_name  \
0   1455458703524700161         Federico     Il_Bale_   
1    822006673267159040       Outsider 🐝  Outsider289   
2              18932422          mizzaro      mizzaro   
3            2478451308     EURO ARTISTS  EUROARTISTS   
4    860108790645084160        Francesco   fra_bomba6   
..                  ...              ...          ...   
91            249984650    Marwah Alaofi     Marwah_k   
92             59780150     Kat Vylomova        ivrik   
93             15068044      Daniel Beck  beck_daniel   
94            748176824            Aleks   aleksbobic   
95            115421781  Debasis Ganguly     debforit   

                          location  followers_count  friends_count  protected  
0                                                16             22      False  
1                                               125            117      False  
2                                               162            358      Fal

In [40]:
print(api.show_friendship('mizzaro'))

TweepError: [{'code': 164, 'message': 'Target user not specified.'}]

In [38]:
#Creazione dataframe degli archi di follow 
def create_edges(follower):
    #aggiunta degli archi
    friendships= []
    for result in friendships[relationship]:
        if(result["source"]["followed_by"]== "true" | result["source"]["following"]== "true" ): 
            if(result["source"]["followed_by"] == true):
                result["source"] = result["source"]["id"]
                result["followed"] = True 
            if(result["source"]["following"] == true):
                result["target"] = result["target"]["id"]
                result["following"] = True 
            friendships.append(result)
    #inserimento nel dataframe
    df_edges = pd.DataFrame(friendships) 
    return df_edges

print(create_edges('mizzaro'))

NameError: name 'relationship' is not defined

# Creazione del grafo

In [41]:
graph = nx.DiGraph(team="Elia Gazzola, Martina Moro, Cristiano Roman, Pierre Agnoletti")

for index,row in df_nodes.iterrows(): 
    twitter_graph.add_node(row["id"],
    id = row["id"],
    title = row["name"],
    color ="#ffff00",
    physics = False, #rende la visualizzazone del grafo più leggera 
    name = row['name'],
    screen_name = row['screen_name'],
    location=row['location'], 
    followers_count = row["followers_count"], 
    following_count = row["following_count"], 
    numbero_of_twitts = row["statuses_count"] #strano nome wtf 
    )
    
#Aggiunta collegamenti al grafo 
for user_id in users_id:#5 profili
    for utente in finale: 
        status=api.show_friendship(id_source=user_id,id_target=utente["id"]) 
        if status[1].following:
            twitter_graph.add_edge(utente["id"],user_id,"follow_rec"=True if status[0].following else False) #nel caso volessimo evidenziare il follow reciproco



SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-41-f4e879c36e59>, line 22)